In [1]:
# Weather Data Collection:

In [2]:
# Configuration 
import configparser

# Create ConfigParser object
config = configparser.ConfigParser()
config.read("Config.ini")

df_1_pest_pkl = config.get("1_PEST", "df_1_pest_pkl")

API_URL = config.get("2_WEATHER", "API_URL")
inputType = config.get("2_WEATHER", "inputType")
outputType = config.get("2_WEATHER", "outputType")
LogIntervalID = config.getint("2_WEATHER", "LogIntervalID")
endDate = config.get("2_WEATHER", "endDate")
nDegrees = config.getint("2_WEATHER", "nDegrees")
lmUser = config.get("2_WEATHER", "lmUser")
lmPass = config.get("2_WEATHER", "lmPass")
AddID = config.getint("2_WEATHER", "AddID")

df_2_weather_pkl = config.get("2_WEATHER", "df_2_weather_pkl")
df_2_weather_csv = config.get("2_WEATHER", "df_2_weather_csv")

In [3]:
import pandas as pd

# unpickle - pandas dataframe
df_pest = pd.read_pickle(df_1_pest_pkl)

print(df_pest.shape)

(530, 20)


In [4]:
df_pest.head()

,delomrade,lan,latitud,longitud,groda,sort,jordart,sadatum,skordear,graderingsdatum,graderingstyp,forforfrukt,forfrukt,ekologisk,plojt,utvecklingsstadium,Bladfläcksvampar,Gulrost,Svartpricksjuka,Sädesbladlus
0,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-01,Veckovis,Vårkorn,Vårkorn,False,True,26,0.0,0.0,0.0,0.0
1,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-08,Veckovis,Vårkorn,Vårkorn,False,True,30,0.0,0.0,0.0,0.0
2,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-15,Veckovis,Vårkorn,Vårkorn,False,True,32,0.0,0.0,0.0,0.0
3,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-22,Veckovis,Vårkorn,Vårkorn,False,True,33,0.0,0.0,0.0,0.0
4,Gotland,Gotlands län,2441000,000,Höstvete,Julius,Lättlera (15-25 % ler),2016-09-18,2017,2017-05-29,Veckovis,Vårkorn,Vårkorn,False,True,37,0.0,0.0,0.0,0.0


In [5]:
import requests
import time
from io import StringIO

t0 = time.time()

# Base API URL
API_URL = "https://www.ffe.slu.se/lm/json/LantmetDWL.cfm"

# Parameters constant across all requests
params = {
    "inputType": "GRID",
    "outputType": "CSV",
    "LogIntervalID": 2,
    "startDate": "2016-01-01",
    "endDate": "2024-12-31",
    "nDegrees": 0,
    "lmUser": "sjvjson",
    "lmPass": "Vjs1017",
    "AddID": 2
}

# List to store weather data
weather_data = []

# Get distinct coordinates:
distinct_coords = df_pest[['latitud', 'longitud']].drop_duplicates()
distinct_coords = distinct_coords.reset_index(drop=True)
total_coords = len(distinct_coords)

# Iterate over each unique coordinate pair
for idx, row in distinct_coords.iterrows():
    latitud = row['latitud']
    longitud = row['longitud']

    # Update API parameters for the current coordinates
    params.update({
        "centerWGS84n": latitud,
        "centerWGS84e": longitud
    })

    # Make the API request
    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status()
        csv_data = response.text

        # Convert CSV text to DataFrame
        df = pd.read_csv(StringIO(csv_data), sep=';')

        # Add latitude and longitude to the dataframe
        df['latitud'] = latitud
        df['longitud'] = longitud

        # Append to weather data list
        weather_data.append(df)
        
        # Print progress
        print(f"Processed {idx + 1}/{total_coords} coordinates (lat={latitud}, lon={longitud}), weather_data_count={len(weather_data)}")

    except requests.RequestException as e:
        print(f"Error fetching data for latitud={latitud}, longitud={longitud}: {e}")

# Combine all dataframes into a single dataframe
df_weather = pd.concat(weather_data, ignore_index=True)

print("Weather data successfully fetched")

# Display the resulting DataFrame
print(df_weather.shape)

t1 = time.time()
print("Seconds Taken:", t1-t0)

Processed 1/10 coordinates (lat=2441000, lon=000), weather_data_count=1
Processed 2/10 coordinates (lat=6135000, lon=397000), weather_data_count=2
Processed 3/10 coordinates (lat=6136000, lon=388000), weather_data_count=3
Processed 4/10 coordinates (lat=6136000, lon=396000), weather_data_count=4
Processed 5/10 coordinates (lat=6138000, lon=391000), weather_data_count=5
Processed 6/10 coordinates (lat=6138000, lon=392000), weather_data_count=6
Processed 7/10 coordinates (lat=6139000, lon=385000), weather_data_count=7
Processed 8/10 coordinates (lat=6139000, lon=400000), weather_data_count=8
Processed 9/10 coordinates (lat=6139000, lon=409000), weather_data_count=9
Processed 10/10 coordinates (lat=6140000, lon=382000), weather_data_count=10
Weather data successfully fetched
(42260, 34)
Seconds Taken: 54.21959924697876


In [6]:
df_weather.head()

,STATIONID,POINTNAME,MUNICID,REGIONNAME,WGS84N,WGS84E,DATAFROM,DATATO,ARABLEPCT,MESANGRIDID,...,WDCM,Q0,BT,G0,G3,G5,G7,G8,G10,WSTNID
0,103357.0,Botkyrka kommun,127.0,Stockholms län,59.140,17.906,20091001.0,20250327.0,10.9,103357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,250317.0,Botkyrka kommun,127.0,Stockholms län,59.100,17.883,20180501.0,20250327.0,40.5,250317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,250945.0,Botkyrka kommun,127.0,Stockholms län,59.127,17.787,20180501.0,20250327.0,49.9,250945.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,252205.0,Botkyrka kommun,127.0,Stockholms län,59.177,17.791,20180501.0,20250327.0,67.0,252205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,252835.0,Botkyrka kommun,127.0,Stockholms län,59.202,17.793,20180501.0,20250327.0,38.4,252835.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42260 entries, 0 to 42259
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   STATIONID    12668 non-null  float64
 1   POINTNAME    12668 non-null  object 
 2   MUNICID      12668 non-null  float64
 3   REGIONNAME   12668 non-null  object 
 4   WGS84N       42260 non-null  float64
 5   WGS84E       42260 non-null  float64
 6   DATAFROM     12668 non-null  float64
 7   DATATO       12668 non-null  float64
 8   ARABLEPCT    12668 non-null  float64
 9   MESANGRIDID  12668 non-null  float64
 10  latitud      42260 non-null  object 
 11  longitud     42260 non-null  object 
 12  DAY          29592 non-null  object 
 13  HOUR         29592 non-null  float64
 14  TM           29592 non-null  float64
 15  TX           29592 non-null  float64
 16  TN           29592 non-null  float64
 17  RR           29592 non-null  float64
 18  RRA          29592 non-null  float64
 19  UM  

In [8]:
# pickle - pandas dataframe
df_weather.to_pickle(df_2_weather_pkl)

# Save the DataFrame to a CSV file with UTF-8 encoding
df_weather.to_csv(df_2_weather_csv, index=False, encoding='utf-8-sig')  # Ensure proper encoding for special characters

print(f"Data pickled to {df_2_weather_pkl} and saved to {df_2_weather_csv}")

Data saved to df_2_weather_t1.csv
